In [19]:
import json
import pandas as pd

with open("./matches/EFBO2372905.json") as f:
    data = f.read()

data = json.loads(data)
match_key = next(iter(data["matchData"][0]["tournamentDatesWithEvents"]))

comp_data = match_data = data["matchData"][0]["tournamentMeta"]
match_data = data["matchData"][0]["tournamentDatesWithEvents"][match_key][0]["events"][0]

In [58]:
match_data.keys()

dict_keys(['eventKey', 'startTime', 'isTBC', 'minutesElapsed', 'minutesIntoAddedTime', 'eventStatus', 'eventStatusNote', 'eventStatusReason', 'eventOutcomeType', 'eventType', 'seriesWinner', 'cpsId', 'cpsLive', 'homeTeam', 'awayTeam', 'eventProgress', 'venue', 'officials', 'tournamentInfo', 'eventActions', 'startTimeInUKHHMM', 'comment', 'href', 'tournamentName', 'tournamentSlug'])

In [59]:
stadium = match_data["venue"]["name"]["full"]
stadium

'Oakwell'

In [60]:
competition = match_data["tournamentName"]["full"]
competition

'Carabao Cup'

In [61]:
game_date = match_data["startTime"][:10]
game_date

'2023-08-08'

In [62]:
ko_time = match_data["startTimeInUKHHMM"]
ko_time

'19:45'

In [65]:
comp_data["tournamentName"]["full"]

'Carabao Cup'

In [ ]:
teams = ["homeTeam", "awayTeam"]
for team in teams:
    team_name = events[team]["name"]["full"]
    try:
        players = events[team]["playerActions"]
        for player in players:
            player_name = action["name"]["full"]
            
            actions = player["actions"]
            for action in actions:
                action_df = pd.json_normalize(action)
                action_df["game_date"] = pd.to_datetime(game_date)
                action_df["team"] = team_name
                action_df["player_name"] = player
                player_actions = pd.concat([player_actions, action_df], axis = 0)
                print(f"Added {game_date}")
    except KeyError:
        pass

In [81]:
player_actions = pd.DataFrame()

teams = ["homeTeam", "awayTeam"]
for team in teams:
    team_name = match_data[team]["name"]["full"]
    try:
        players = match_data[team]["playerActions"]
        for player in players:
            player_name = player["name"]["full"]
            print(player_name)
            
            actions = player["actions"]
            for action in actions:
                action_df = pd.json_normalize(action)
                action_df["game_date"] = pd.to_datetime(game_date)
                action_df["team"] = team_name
                action_df["player_name"] = player_name
                player_actions = pd.concat([player_actions, action_df], axis = 0)
    except KeyError:
        pass

Herbie Kane
Aiden Marsh
Luke Norris
Samuel Taylor


In [115]:
goals  = player_actions.query("type == 'goal' & team == 'Tranmere Rovers'") \
    .rename(columns = {
        "timeElapsed": "goal_min",
        "ownGoal": "own_goal"
    }).sort_values("goal_min") \
    .drop_duplicates()

goals.penalty = goals.penalty.astype(int)
goals.own_goal = goals.own_goal.astype(int)
goals.loc[goals.own_goal == 1, "player_name"] = "OG"

goals = goals[["game_date", "player_name", "goal_min", "penalty", "own_goal"]]
if not goals.empty:
    # goals.to_csv("goals_updates.csv", index = False)
    print(f"{len(goals)} goals saved.")
else:
    print("No goals found.")

goals

2 goals saved.


,game_date,player_name,goal_min,penalty,own_goal
0,2023-08-08,Luke Norris,23,0,0
0,2023-08-08,Samuel Taylor,47,0,0


In [110]:
red_cards = player_actions.query("type.str.contains('red-card')") \
    .rename(columns = {
        "timeElapsed": "min_so",
    }).drop_duplicates()

red_cards = red_cards[["game_date", "player_name", "min_so"]]
if not red_cards.empty:
    red_cards.to_csv("red_card_updates.csv", index = False)
else:
    print("No red cards found.")

red_cards

,game_date,player_name,min_so


In [101]:
bbc = pd.read_csv("~/Desktop/bbc-goals-and-cards.csv").query("type.str.contains('red-card')").drop_duplicates()

bbc

,type,timeElapsed,addedTime,penalty,ownGoal,displayTime,game_date,team,player_name
41,yellow-red-card,73,0,NaN,NaN,73',2012-10-13 15:00:00+01:00,Yeovil Town,Joe Edwards
78,red-card,36,0,NaN,NaN,36',2013-02-02 15:00:00+00:00,Tranmere Rovers,Jean-Louis Akpa Akpro
83,red-card,6,0,NaN,NaN,6',2013-02-15 19:45:00+00:00,Tranmere Rovers,Liam Palmer
129,yellow-red-card,33,0,NaN,NaN,33',2013-08-17 15:00:00+01:00,Tranmere Rovers,Danny Holmes
131,red-card,38,0,NaN,NaN,38',2013-08-24 15:00:00+01:00,Tranmere Rovers,Andrai Jones
...,...,...,...,...,...,...,...,...,...
1510,red-card,69,0,NaN,NaN,69',2022-10-04 19:00:00+01:00,Leeds United U21,Sam Greenwood
1528,red-card,90,5,NaN,NaN,90'+5,2022-11-12 15:00:00+00:00,Tranmere Rovers,Jordan Turnbull
1529,red-card,30,0,NaN,NaN,30',2022-11-19 15:00:00+00:00,Tranmere Rovers,Dynel Simeu
1542,red-card,76,0,NaN,NaN,76',2022-12-29 19:45:00+00:00,Tranmere Rovers,Paul Lewis
